In [2]:
!pip install pandas
import pandas as pd
import requests
pd.set_option('display.float_format', lambda x: '%.3f' % x)

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement urllib2
ERROR: No matching distribution found for urllib2
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
# Bearwhale wallet v2 : 9eyXNatnA6YM4tS1TjadEA6TFrd9bdufbFuykV89iX9vE9RBZZe
# v1 : 9hyDXH72HoNTiG2pvxFQwxAhWBU8CrbvwtJDtnYoa4jfpaSk1d3
target = '9eyXNatnA6YM4tS1TjadEA6TFrd9bdufbFuykV89iX9vE9RBZZe'

In [4]:
# API Request
url = 'https://api.ergoplatform.com/api/v1/boxes/byAddress/'+target
r = requests.get(url)
json = r.json()

In [5]:
# Query the json directly

#json.keys()
#json['items']
#json['total']
#json

In [6]:
# load json into a dataframe
df = pd.DataFrame(json['items'])

In [ ]:
df.info()

# boxId
# transactionId - tx id displayed on explorer
# blockId
# value - unspent 
# index - ? 
# creationHeight - time
# ergoTree -  0008cd023c166f1ba9de6fdd5ed90a6f15e037b0a9247a...
# address - wallet address
# assets - tokens
# additionalRegisters - sigma? R4
# spentTxId
# mainChain - main-net - True

In [7]:
# Drop unneeded columns
df.drop(columns=['mainChain','ergoTree','transactionId','boxId', 
                 'blockId','address','index',
                 'additionalRegisters','spentTransactionId'], inplace=True)

In [8]:
# Finding difference
df['BalanceChange (ERG)'] = df.value.diff()
df['HeightChange (h)'] = df.creationHeight.diff()
# nanoErg -> Erg
df['value'] = df['value'] / 1000000000
df['BalanceChange (ERG)'] = df['BalanceChange (ERG)'] / 1000000000
# Track this from geneis block ?
df['HeightChange (h)'] = df['HeightChange (h)'] * 2
df['HeightChange (h)'] = df['HeightChange (h)'] / 60
# Reversing polarity
df['BalanceChange (ERG)'] = df['BalanceChange (ERG)'] * -1 
df['HeightChange (h)'] = df['HeightChange (h)'] * -1 
df['BalanceChange (ERG)'] = df['BalanceChange (ERG)'].shift(-1)
df['HeightChange (h)'] = df['HeightChange (h)'].shift(-1)

In [9]:
df3 = df['assets'].apply(pd.Series)
df4 = df3[0].apply(pd.Series)
df4.drop(columns=['tokenId','index','decimals','type', 
                 ], inplace=True)



In [10]:
df = pd.concat([df, df4], axis=1)
df.drop(columns=['assets'], inplace=True)
df = df.rename(columns={'value': 'Final Balance', 'amount': 'Sigma', 'name': 'type'})
df


,Final Balance,creationHeight,BalanceChange (ERG),HeightChange (h),0,Sigma,type
0,8396.648,481912,-52939.309,9.267,nan,19991685.000,SigUSD
1,61335.956,481634,52551.553,10.867,nan,nan,NaN
2,8784.404,481308,-2754.760,1.400,nan,19364635.000,SigUSD
3,11539.163,481266,-6278.562,2.033,nan,18325836.000,SigUSD
4,17817.726,481205,-2913.262,5.100,nan,15969891.000,SigUSD
5,20730.988,481052,-40237.476,45.300,nan,14887354.000,SigUSD
6,60968.464,479693,22279.442,26.967,nan,nan,NaN
7,38689.022,478884,-22367.429,13.000,nan,8008643.000,SigUSD
8,61056.450,478494,48684.407,9.333,nan,nan,NaN
9,12372.043,478214,-47952.661,1.767,nan,17536327.000,SigUSD


In [11]:
df['BalanceChange (ERG)'].sum()

4144.1438561090035

In [12]:
df['Final Balance'].sum()

428928.286749852